<a href="https://colab.research.google.com/github/ssook1222/bigdata-analysist-certificate/blob/master/bigdata-analysist-certificate/task2/service_exited.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/churn-model-data-set-competition-form/X_test.csv
/kaggle/input/churn-model-data-set-competition-form/y_train.csv
/kaggle/input/churn-model-data-set-competition-form/X_train.csv
/kaggle/input/churn-model-data-set-competition-form/test_label/y_test.csv


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score

In [ ]:
X_train  = pd.read_csv('/kaggle/input/churn-model-data-set-competition-form/X_train.csv')
X_test  = pd.read_csv('/kaggle/input/churn-model-data-set-competition-form/X_test.csv')

y_train  = pd.read_csv('/kaggle/input/churn-model-data-set-competition-form/y_train.csv')
y_test = pd.read_csv('/kaggle/input/churn-model-data-set-competition-form/test_label/y_test.csv')

In [ ]:
le = LabelEncoder()
X_train['Geography'] = le.fit_transform(X_train['Geography'])
X_test['Geography'] = le.fit_transform(X_test['Geography'])

X_train['Gender'] = le.fit_transform(X_train['Gender'])
X_test['Gender'] = le.fit_transform(X_test['Gender'])

In [ ]:
X_train = X_train[['CustomerId', 'CreditScore', 'Geography', 'Gender', 'Age','Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember','EstimatedSalary']]
X_test =  X_test[['CustomerId', 'CreditScore', 'Geography', 'Gender', 'Age','Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember','EstimatedSalary']]

y_train = y_train.sort_values('CustomerId')
X_train = X_train.sort_values('CustomerId')

y_test = y_test.sort_values('CustomerId')
X_test = X_test.sort_values('CustomerId')

y_train = y_train['Exited']
y_test = y_test['Exited']

In [ ]:
X_train.columns

Index(['CustomerId', 'CreditScore', 'Geography', 'Gender', 'Age', 'Tenure',
       'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember',
       'EstimatedSalary'],
      dtype='object')

In [ ]:
scale_list = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']

scaler = StandardScaler()
X_train[scale_list] = scaler.fit_transform(X_train[scale_list])
X_test[scale_list] = scaler.transform(X_test[scale_list])

In [ ]:
rf = RandomForestClassifier(random_state=1)

In [ ]:
params = {"n_estimators":[100,200,300,400,500], "max_depth":[1,2,3,4,5]}
gs = GridSearchCV(rf, param_grid=params, cv = 5, scoring="roc_auc", n_jobs=4)

In [ ]:
gs.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=1), n_jobs=4,
             param_grid={'max_depth': [1, 2, 3, 4, 5],
                         'n_estimators': [100, 200, 300, 400, 500]},
             scoring='roc_auc')

In [ ]:
print(f"score : {gs.best_score_}")

score : 0.8473714531146852


In [ ]:
test_predict = gs.predict(X_test)
test_predict_prob = gs.predict_proba(X_test)

In [ ]:
print(f"accuracy_score:", accuracy_score(y_test, test_predict))
print(f"roc_auc_score:", roc_auc_score(y_test, test_predict_prob[:,1]))

accuracy_score: 0.8546129677235076
roc_auc_score: 0.8462318974728399


In [ ]:
submit = pd.DataFrame({"CustomerId": X_test['CustomerId'], "Exited" : test_predict_prob[:,1]})
print(submit.head(5))

      CustomerId    Exited
2250    15565714  0.229153
1398    15565796  0.564313
2920    15565879  0.079852
1274    15566251  0.165040
2005    15566269  0.076956


In [ ]:
submit.to_csv("result.csv", index = False)